In [6]:
import pandas as pd

from IPython.display import clear_output
import numpy as np
import datetime

from termcolor import colored

In [33]:
# ==============================flair models=================================
from flair.data import Sentence
from flair.models import SequenceTagger
# load tagger
tagger = SequenceTagger.load("flair/pos-english")

# ==========================transformers models=============================
from transformers import pipeline,AutoTokenizer, AutoModelForSeq2SeqLM,AutoModelForQuestionAnswering

# classifier_zero_shot_cf = pipeline("zero-shot-classification",
#                       model="facebook/bart-large-mnli")

# Q-A(blenderbot)
tokenizer_QA = AutoTokenizer.from_pretrained("facebook/blenderbot-400M-distill")
model_QA = AutoModelForSeq2SeqLM.from_pretrained("facebook/blenderbot-400M-distill")

# Summarization
summarizer = pipeline("summarization", model="google/pegasus-xsum")

# Q-A(roberta)
model_name = "deepset/roberta-base-squad2"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

# sentiment -distilbert-base-uncased-finetuned-sst-2-english
classifier_sentiment = pipeline('sentiment-analysis', model="distilbert-base-uncased-finetuned-sst-2-english")


# ======================sentence_transformers models=========================
from sentence_transformers import SentenceTransformer, util
model_sentence_sim = SentenceTransformer('all-MiniLM-L6-v2')

2021-12-05 14:27:19,658 loading file /Users/cholian/.flair/models/pos-english/a9a73f6cd878edce8a0fa518db76f441f1cc49c2525b2b4557af278ec2f0659e.121306ea62993d04cd1978398b68396931a39eb47754c8a06a87f325ea70ac63


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [237]:
import requests
from urllib.parse import urljoin
import bs4

import time

In [343]:
# search the keywords in google
def google_search(tokens):
    
    text= f"What is {tokens}?"
    url = 'https://google.com/search?q=' + text
    
    # Fetch the URL data using requests.get(url),
    # store it in a variable, request_result.
    request_result=requests.get(url)
    
    # Creating soup from the fetched request
    soup = bs4.BeautifulSoup(request_result.text,
                            "html.parser")
    time.sleep(0.5)
    
    all_result = soup.find_all('div', class_="BNeawe s3v9rd AP7Wnd")
    context = ".".join([all_result[i].text for i in range(10)])
    
    return context

# suummarize the google search result
def summary_context(tokens,context):
    context = f'The definition of {tokens} is that:'+context

    summary_text = summarizer(context, max_length=300, min_length=10, do_sample=False)[0]['summary_text']
    return summary_text

# make sure whether the token is related to the crypto
def final_sure(tokens,summary_text):

    related_list = ['bitcoin','btc','ethereum','crypto','cryptocurrency','blockchain','digitalcoin','digital currency','digital asset','digital']
    def_corpus = f"The definication of {tokens} is related to: "+" or ".join(related_list)
    sentences = [def_corpus, summary_text]

    #Encode all sentences
    embeddings = model_sentence_sim.encode(sentences)

    #Compute cosine similarity between all pairs
    cos_sim = util.cos_sim(embeddings, embeddings)

    sim_score = float(cos_sim[0][1])

    sim_thred = 0.5
    if sim_score>sim_thred:
        return True
    else:
        return False
    
# select tokens that only are related to the crypto 
def token_is_crypto(tag_text):
    crypto_tokens = []
    for i in range(0, len(tag_text)):
        tokens = tag_text[i]
        
        context = google_search(tokens)
        
        summary_text = summary_context(tokens,context)

        if final_sure(tokens,summary_text):
            crypto_tokens.append(tokens)
    return crypto_tokens
    
# for i in range(len(title)):

def labeled_title(each_title,each__time):

    t = datetime.datetime.strptime(each_time,"%Y-%m-%dT%H:%M:%SZ")
    t.strftime("%Y-%m-%d %H:%M:%S")
    
    # make sentence
    sentence = Sentence(each_title)

    # predict NER tags
    tagger.predict(sentence)

    all_tag = np.array([i.tag for i in sentence.get_spans('pos')])
    all_text = np.array([i.text for i in sentence.get_spans('pos')])
        
    tag_text = []
    import_lb = ['NNP','NNPS',"NN"]

    for _tag,_text in zip(all_tag,all_text):
        if _tag in import_lb:
            tag_text.append(_text)
            
    crypto_tokens = token_is_crypto(tag_text)
    
    print(colored("Time:",'yellow'), t)
    print(colored("News title: ",'yellow'),each_title)
    print("Keywords: ",tag_text)
    print(colored("Crypto Label: ",'yellow'),colored(crypto_tokens,'green'))
    print("--------------------------------------------------")
    # clear_output(wait=True)

In [222]:
crypto_tokens = token_is_crypto(tag_text)

Your max_length is set to 300, but you input_length is only 187. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


In [ ]:
ori_data = pd.read_csv('sample_cryoto_news.csv',index_col=0)

title = ori_data['title']
create_time = ori_data['created_at']

In [326]:
# 4,
i = 5
each_title = ori_data['title'][i]
each_time = ori_data['created_at'][i]
labeled_title(each_title,each_time)

Your max_length is set to 300, but you input_length is only 47. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 300, but you input_length is only 64. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 300, but you input_length is only 101. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 300, but you input_length is only 41. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 300, but you input_length is only 49. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Time: 2021-12-04 12:30:05
News title:  Bitcoin, Ethereum face largest correction since 19 May; is it time to buy the dip
Keywords:  ['Bitcoin', 'Ethereum', 'face', 'correction', 'May', 'time', 'dip']
Crypto Label:  ['Bitcoin', 'Ethereum', 'face', 'correction', 'May', 'time', 'dip']
--------------------------------------------------


In [351]:
tokens = "Ethereum"

In [352]:
context = google_search(tokens)      
summary_text = summary_context(tokens,context)

In [353]:
context
summary_text

'Ethereum is a blockchain platform with its own cryptocurrency, Ether (ETH) or Ethereum, and its own programming language, called Solidity.'

In [354]:
related_list = ['bitcoin','btc','ethereum','crypto','cryptocurrency','blockchain','digitalcoin','digital currency','digital asset']
def_corpus = f"The definication is that: "+" or ".join(related_list)
sentences = [def_corpus, summary_text]

#Encode all sentences
embeddings = model_sentence_sim.encode(sentences)

#Compute cosine similarity between all pairs
cos_sim = util.cos_sim(embeddings, embeddings)

sim_score = float(cos_sim[0][1])

In [355]:
sim_score

0.5591626167297363